In [ ]:
pip install dash pandas plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
pip install dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
# Layout for periodic updates
app.layout = html.Div([
    html.H1("Real-Time Social Media Dashboard", style={'textAlign': 'center'}),

    dcc.Interval(
        id='interval-update',
        interval=60 * 1000,  # 1-minute interval for data refresh
        n_intervals=0
    ),

    # Same layout as before...
])

# Callback to update the graph and table periodically
@app.callback(
    [Output('engagement-graph', 'figure'),
     Output('top-posts-table', 'children')],
    [Input('subreddit-dropdown', 'value'),
     Input('interval-update', 'n_intervals')]
)
def update_dashboard(subreddit_name, n_intervals):
    # Same code to fetch and update data as before...
    pass


In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import praw
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output

client_id = 'SwiZpLVMOf9z_XTpJCF0pA'
client_secret = 'L2xq30TATixyLbVAgO3VdD1YNchU2A'
user_agent = 'MyRedditApp/1.0 by KnownComplaint82'

# Create a Reddit instance
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Dash app setup
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Fetch top posts from a subreddit (e.g., 'Python')
def get_top_posts(subreddit_name="Python", limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts = list(subreddit.hot(limit=limit))

    post_data = []
    for post in posts:
        post_data.append({
            "title": post.title,
            "score": post.score,
            "num_comments": post.num_comments,
            "engagement": post.score + post.num_comments,
            "author": str(post.author),
            "url": post.url
        })
    return pd.DataFrame(post_data)

# Layout for the Dashboard
app.layout = html.Div([
    html.H1("Real-Time Social Media Dashboard", style={'textAlign': 'center'}),

    # Dropdown for subreddit selection
    html.Div([
        html.Label("Select Subreddit"),
        dcc.Dropdown(
            id='subreddit-dropdown',
            options=[
                {'label': 'Python', 'value': 'Python'},
                {'label': 'MachineLearning', 'value': 'MachineLearning'},
                {'label': 'DataScience', 'value': 'DataScience'},
                {'label': 'Technology', 'value': 'Technology'},
            ],
            value='Python',  # Default value
            style={'width': '50%'}
        ),
    ], style={'padding': '20px', 'width': '50%', 'margin': 'auto'}),

    # Graph to display top posts engagement
    dcc.Graph(id='engagement-graph'),

    # Display a table for top 5 posts
    html.Div([
        html.H4("Top 5 Posts"),
        dbc.Table(id="top-posts-table")
    ])
])

# Callback to update graph and table based on selected subreddit
@app.callback(
    [Output('engagement-graph', 'figure'),
     Output('top-posts-table', 'children')],
    [Input('subreddit-dropdown', 'value')]
)
def update_dashboard(subreddit_name):
    # Get the top posts for the selected subreddit
    df = get_top_posts(subreddit_name)

    # Create a bar chart of top posts engagement
    fig = px.bar(df, x='title', y='engagement', title=f"Top Posts Engagement in r/{subreddit_name}",
                 labels={'title': 'Post Title', 'engagement': 'Engagement (Score + Comments)'})

    # Create the table for top 5 posts
    table_header = [html.Tr([html.Th("Title"), html.Th("Score"), html.Th("Num Comments"), html.Th("Engagement")])]
    table_body = [html.Tr([html.Td(df.iloc[i]['title']),
                          html.Td(df.iloc[i]['score']),
                          html.Td(df.iloc[i]['num_comments']),
                          html.Td(df.iloc[i]['engagement'])]) for i in range(min(5, len(df)))]

    return fig, table_header + table_body


# Run the app
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>